In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[2]').getOrCreate()

In [2]:
hdfs_path = "hdfs://hdfs-nn:9000/project/Bronze/Demography/US County Census.csv"

In [3]:
#census = spark.read.csv(hdfs_path)
census = spark.read.option("header", "true").csv(hdfs_path)
#por o cabeçalho do csv como cabeçalho aqui

In [4]:
census.printSchema()
census.show()
census.toPandas()

root
 |-- FIPS: string (nullable = true)
 |-- STFIPS: string (nullable = true)
 |-- COFIPS: string (nullable = true)
 |-- state_abbrev: string (nullable = true)
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- year: string (nullable = true)
 |-- pop: string (nullable = true)
 |-- white_pop: string (nullable = true)
 |-- black_pop: string (nullable = true)
 |-- asian_pop: string (nullable = true)
 |-- indian_pop: string (nullable = true)
 |-- pacific_pop: string (nullable = true)
 |-- two_pop: string (nullable = true)
 |-- not_hisp_pop: string (nullable = true)
 |-- hisp_pop: string (nullable = true)

+-----+------+------+------------+-------+-------+----+------+---------+---------+---------+----------+-----------+-------+------------+--------+
| FIPS|STFIPS|COFIPS|state_abbrev|  state| county|year|   pop|white_pop|black_pop|asian_pop|indian_pop|pacific_pop|two_pop|not_hisp_pop|hisp_pop|
+-----+------+------+------------+-------+-------+----+------+-------

,FIPS,STFIPS,COFIPS,state_abbrev,state,county,year,pop,white_pop,black_pop,asian_pop,indian_pop,pacific_pop,two_pop,not_hisp_pop,hisp_pop
0,01001,01,001,AL,Alabama,Autauga,2010,54571,43297,9689,484,258,47,796,53261,1310
1,01001,01,001,AL,Alabama,Autauga,2011,55227,43699,9883,514,261,51,819,53888,1339
2,01001,01,001,AL,Alabama,Autauga,2012,54954,43315,9949,552,275,44,819,53639,1315
3,01001,01,001,AL,Alabama,Autauga,2013,54727,42943,9984,561,279,46,914,53363,1364
4,01001,01,001,AL,Alabama,Autauga,2014,54893,42945,10103,573,279,50,943,53461,1432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31405,56045,56,045,WY,Wyoming,Weston,2015,7208,6835,39,81,107,2,144,6923,285
31406,56045,56,045,WY,Wyoming,Weston,2016,7220,6826,38,88,108,2,158,6924,296
31407,56045,56,045,WY,Wyoming,Weston,2017,6968,6558,44,97,114,2,153,6681,287
31408,56045,56,045,WY,Wyoming,Weston,2018,6924,6474,47,109,125,2,167,6651,273


In [5]:
#mudar de string para int
from pyspark.sql.types import IntegerType
census = census.withColumn("year", census["year"].cast(IntegerType()))
census = census.withColumn("pop", census["pop"].cast(IntegerType()))
census = census.withColumn("white_pop", census["white_pop"].cast(IntegerType()))
census = census.withColumn("black_pop", census["black_pop"].cast(IntegerType()))
census = census.withColumn("asian_pop", census["asian_pop"].cast(IntegerType()))
census = census.withColumn("indian_pop", census["indian_pop"].cast(IntegerType()))
census = census.withColumn("pacific_pop", census["pacific_pop"].cast(IntegerType()))
census = census.withColumn("two_pop", census["two_pop"].cast(IntegerType()))
census.printSchema()

root
 |-- FIPS: string (nullable = true)
 |-- STFIPS: string (nullable = true)
 |-- COFIPS: string (nullable = true)
 |-- state_abbrev: string (nullable = true)
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- pop: integer (nullable = true)
 |-- white_pop: integer (nullable = true)
 |-- black_pop: integer (nullable = true)
 |-- asian_pop: integer (nullable = true)
 |-- indian_pop: integer (nullable = true)
 |-- pacific_pop: integer (nullable = true)
 |-- two_pop: integer (nullable = true)
 |-- not_hisp_pop: string (nullable = true)
 |-- hisp_pop: string (nullable = true)



In [6]:
columns_to_drop = ["FIPS", "STFIPS", "COFIPS","county", "not_hisp_pop", "hisp_pop"]
census = census.drop(*columns_to_drop)

In [7]:
#mudar o nome das colunas
census = census.withColumnRenamed("state_abbrev", "state_abb")
census = census.withColumnRenamed("pop", "total_pop")
census = census.withColumnRenamed("black_pop", "black")
census = census.withColumnRenamed("white_pop", "white")
census = census.withColumnRenamed("indian_pop", "native")
census.toPandas()

,state_abb,state,year,total_pop,white,black,asian_pop,native,pacific_pop,two_pop
0,AL,Alabama,2010,54571,43297,9689,484,258,47,796
1,AL,Alabama,2011,55227,43699,9883,514,261,51,819
2,AL,Alabama,2012,54954,43315,9949,552,275,44,819
3,AL,Alabama,2013,54727,42943,9984,561,279,46,914
4,AL,Alabama,2014,54893,42945,10103,573,279,50,943
...,...,...,...,...,...,...,...,...,...,...
31405,WY,Wyoming,2015,7208,6835,39,81,107,2,144
31406,WY,Wyoming,2016,7220,6826,38,88,108,2,158
31407,WY,Wyoming,2017,6968,6558,44,97,114,2,153
31408,WY,Wyoming,2018,6924,6474,47,109,125,2,167


In [8]:
census = census.withColumn("asian_or_pacific_islander", col("asian_pop")+col("pacific_pop"))
columns_to_drop = ["asian_pop", "pacific_pop"]
census = census.drop(*columns_to_drop)
census.printSchema()
census.toPandas()

root
 |-- state_abb: string (nullable = true)
 |-- state: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- total_pop: integer (nullable = true)
 |-- white: integer (nullable = true)
 |-- black: integer (nullable = true)
 |-- native: integer (nullable = true)
 |-- two_pop: integer (nullable = true)
 |-- asian_or_pacific_islander: integer (nullable = true)



,state_abb,state,year,total_pop,white,black,native,two_pop,asian_or_pacific_islander
0,AL,Alabama,2010,54571,43297,9689,258,796,531
1,AL,Alabama,2011,55227,43699,9883,261,819,565
2,AL,Alabama,2012,54954,43315,9949,275,819,596
3,AL,Alabama,2013,54727,42943,9984,279,914,607
4,AL,Alabama,2014,54893,42945,10103,279,943,623
...,...,...,...,...,...,...,...,...,...
31405,WY,Wyoming,2015,7208,6835,39,107,144,83
31406,WY,Wyoming,2016,7220,6826,38,108,158,90
31407,WY,Wyoming,2017,6968,6558,44,114,153,99
31408,WY,Wyoming,2018,6924,6474,47,125,167,111


In [9]:
census = census.groupby('state_abb','state','year').agg(
    sum("total_pop").alias("total_pop"),
    sum("white").alias("white"),
    sum("black").alias("black"),
    sum('native').alias('native'),
    sum("two_pop").alias("two_pop"),
    sum("asian_or_pacific_islander").alias("asian_or_pacific_islander")
)
census.toPandas()

,state_abb,state,year,total_pop,white,black,native,two_pop,asian_or_pacific_islander
0,MN,Minnesota,2015,5482032,4674959,336891,73976,130127,266079
1,IL,Illinois,2014,12884493,9983837,1895699,74245,235714,694998
2,SC,South Carolina,2013,4764080,3255341,1330419,25525,78521,74274
3,VT,Vermont,2015,625216,592805,7836,2365,11423,10787
4,HI,Hawaii,2015,1422052,370640,29752,5637,334635,681388
...,...,...,...,...,...,...,...,...,...
495,NH,New Hampshire,2018,1353465,1262037,23338,4068,23577,40445
496,NM,New Mexico,2014,2089568,1728148,51259,221635,50329,38197
497,ND,North Dakota,2017,754942,660053,23603,42067,16133,13086
498,OR,Oregon,2017,4143625,3608387,89388,74883,157757,213210


In [10]:
census = census.select("state_abb","state","total_pop","white","black","native","two_pop","asian_or_pacific_islander","year")

census = census.withColumn("total_pop", census["total_pop"].cast(IntegerType()))
census = census.withColumn("white", census["white"].cast(IntegerType()))
census = census.withColumn("black", census["black"].cast(IntegerType()))
census = census.withColumn("native", census["native"].cast(IntegerType()))
census = census.withColumn("two_pop", census["two_pop"].cast(IntegerType()))
census = census.withColumn("asian_or_pacific_islander", census["asian_or_pacific_islander"].cast(IntegerType()))

census.toPandas()


,state_abb,state,total_pop,white,black,native,two_pop,asian_or_pacific_islander,year
0,MN,Minnesota,5482032,4674959,336891,73976,130127,266079,2015
1,IL,Illinois,12884493,9983837,1895699,74245,235714,694998,2014
2,SC,South Carolina,4764080,3255341,1330419,25525,78521,74274,2013
3,VT,Vermont,625216,592805,7836,2365,11423,10787,2015
4,HI,Hawaii,1422052,370640,29752,5637,334635,681388,2015
...,...,...,...,...,...,...,...,...,...
495,NH,New Hampshire,1353465,1262037,23338,4068,23577,40445,2018
496,NM,New Mexico,2089568,1728148,51259,221635,50329,38197,2014
497,ND,North Dakota,754942,660053,23603,42067,16133,13086,2017
498,OR,Oregon,4143625,3608387,89388,74883,157757,213210,2017


In [11]:
census = census.withColumn("year",col("year").cast("string"))
census.show()

+---------+--------------+---------+-------+-------+------+-------+-------------------------+----+
|state_abb|         state|total_pop|  white|  black|native|two_pop|asian_or_pacific_islander|year|
+---------+--------------+---------+-------+-------+------+-------+-------------------------+----+
|       MN|     Minnesota|  5482032|4674959| 336891| 73976| 130127|                   266079|2015|
|       IL|      Illinois| 12884493|9983837|1895699| 74245| 235714|                   694998|2014|
|       SC|South Carolina|  4764080|3255341|1330419| 25525|  78521|                    74274|2013|
|       VT|       Vermont|   625216| 592805|   7836|  2365|  11423|                    10787|2015|
|       HI|        Hawaii|  1422052| 370640|  29752|  5637| 334635|                   681388|2015|
|       WI|     Wisconsin|  5751525|5048909| 379440| 64172| 101456|                   157548|2014|
|       MN|     Minnesota|  5566230|4700264| 368596| 76089| 138003|                   283278|2017|
|       CT

In [12]:
from pyspark.sql.types import *
census = census.withColumn("year", census["year"].cast(DateType()))

In [13]:

census.printSchema()

root
 |-- state_abb: string (nullable = true)
 |-- state: string (nullable = true)
 |-- total_pop: integer (nullable = true)
 |-- white: integer (nullable = true)
 |-- black: integer (nullable = true)
 |-- native: integer (nullable = true)
 |-- two_pop: integer (nullable = true)
 |-- asian_or_pacific_islander: integer (nullable = true)
 |-- year: date (nullable = true)



In [14]:
census.show()

+---------+--------------+---------+-------+-------+------+-------+-------------------------+----------+
|state_abb|         state|total_pop|  white|  black|native|two_pop|asian_or_pacific_islander|      year|
+---------+--------------+---------+-------+-------+------+-------+-------------------------+----------+
|       MN|     Minnesota|  5482032|4674959| 336891| 73976| 130127|                   266079|2015-01-01|
|       IL|      Illinois| 12884493|9983837|1895699| 74245| 235714|                   694998|2014-01-01|
|       SC|South Carolina|  4764080|3255341|1330419| 25525|  78521|                    74274|2013-01-01|
|       VT|       Vermont|   625216| 592805|   7836|  2365|  11423|                    10787|2015-01-01|
|       HI|        Hawaii|  1422052| 370640|  29752|  5637| 334635|                   681388|2015-01-01|
|       WI|     Wisconsin|  5751525|5048909| 379440| 64172| 101456|                   157548|2014-01-01|
|       MN|     Minnesota|  5566230|4700264| 368596| 76

In [15]:
census \
    .repartition(1) \
    .write \
    .partitionBy("year") \
    .format("parquet") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/warehouse/americancrimes.db/demography")

